 # Création de la dataframe pour les actes de dialogues dans Snorkel
 

In [1]:
import sys
sys.path.insert(1,"scripts/")
from alignments import *
from tony import *
from gold import *
from silences import *
from speaker import *
from tony import *
from gold import *
from punctuation import *
from pitchenergy import *
from punctuation_samir import *

In [2]:
#Emplacement des fichiers : 

name_meeting="Linagora_R1"
file_alignments="data/alignments/Linagora_R1_align.txt"
#file_spk_change="data/speakers/Linagora_R1_spk.txt"
file_spk_change="test_loc/Linagora_R1.txt"
file_silences="data/silences/Linagora_R1_sil.txt"
file_tony_result="data/tony/Linagora_R1_tony.txt.split.tok"
file_gold="data/gold/Linagora_R1_gold.txt"
file_punct="data/punctuation/Linagora_R1_punct.txt"
file_punct_samir="data/punctuation_samir/Linagora_R1_samir.txt"
file_pitchenergy="data/audio/Linagora_R1_pitchenergy.txt"

    
work_directory="path"


## Fonctions

### Les fichiers nécessaires 
<br>

#### 1. Fichiers d'alignements
<br>
Dans la réalité il suffira d'obtenir le résultat de la reconnaissance vocale, qui doit normalement fournir le mot transcrit, le début du mot en secondes et la fin du mot en secondes.
Pour l'étude, il s'agit de la sortie du logiciel Jtrans en y mettant en entrée l'audio et le fichier .trs correspondant contenant la transcription faite par Authôt.
<br>


#### 2. Fichiers de détection de changement de locuteur 
<br>
Il s'agit des fichiers de sortie de "pyannote_SCD_OVL.ipynb" qui prend en entrée un fichier wav et donne en sortie un fichier avec des segments en format hh:mm:ss de même locuteur. Il faut convertir ce fichier en segement en secondes.
<br>

#### 3. Fichier des segments de silence

Fichiers issus de l'utilisation de Py WebRTCVad, qui ressemble à ça: <br>
<br>
*File : Linagora_A1_0_05_27--end.wav<br>
0.09 0.30 0.18 NS<br>
0.48 0.63 0.12 NS<br>
...*<br>
<br>
Les trois premières colonnes correspondent à :
Debut/Fin/Durée-3s/NS
<br>
#### 4. Fichier sortie de ToNy  
<br>
Fichiers issus de l'utilisation du système ToNy. Des fonctions ont été conçues pour créer les fichiers nécessaires à son utilisation. <br>
Les fichiers de sortie de ToNy ressemblent à ça :<br>
<br>
*1	eh	_	_	_	_	_	_	_	BeginSeg=Yes<br>
2	ben	_	_	_	_	_	_	_	_<br>
3	du	_	_	_	_	_	_	_	_<br>
4	coup	_	_	_	_	_	_	_	_<br>
5	ouais	_	_	_	_	_	_	_	_*<br>


#### 5. Les fichiers Gold 
<br>
Pour les réunions qui ont des labels GOLD, donner les fichiers textes avec les séparateurs | d'actes de dialogues
<br>

## Liste des fonctions 
<br>

### Alignements


**alignments_word_extraction(filename_alignments)**
> return **word, beg_word, end_word** 
> respectivement la liste des mots, le début des mots en secondes, la fin des mots en secondes 
<br>

**alignments_real_round_extraction(filename_alignments)**
> return n_round, beg_round, end_round, rank_round, text_round, loc


<br>


### Détection changement de locuteurs

<br>

**read_change_spk_detection(filename_change_spk_detection)**
> return **beg_chg_spk**, **end_chg_spk**


**round_extraction(word, beg_word, end_word, beg_chg_spk ,end_chg_spk)**
> return **n_round**, **beg_round**, **end_round**, **rank_round**, **text_round**

<br>

### Silences

<br>

**extract_silences_positions(filename_silences)**
> return **beg_sil**, **end_sil**, **dur_sil**, **mid_sil**


**silences_word_position(word, beg_word, end_word, beg_sil, end_sil, dur_sil, mid_sil)**
> return **sil_bef**, **sil_aft**

<br>

### ToNy 

<br>

**data_prep_tony(name, word, rank_round, text_round)**
> create file for ToNy


**convert_tony_results(filename_tony_result)**
> return **tony**

<br>


### Gold

<br>

**convert_gold(filename_gold)**
> return **gold**


<br>

### Ponctuation

<br>

**convert_punctuation(filename_punct)**
> return **prob_nothing**, **prob_point**, **prob_comma**, **prob_nothing_bef**, **prob_point_bef**, **prob_comma_bef**


<br>

**punctuation_samir_extraction(file_punctuation, word)**
> return **punct**, **punct_bef**



### Creation pour une seule réunion

##### Alignements

In [3]:
word,beg_word,end_word = alignments_word_extraction(file_alignments)
print(len(word), len(beg_word), len(end_word))

7943 7943 7943


In [4]:
real_n_round, real_beg_round, real_end_round, real_rank_round, real_text_round, real_loc=alignments_real_round_extraction(file_alignments)

7943 7943 7943 7943 7943


##### Silences

In [5]:

beg_sil,end_sil,dur_sil,mid_sil=extract_silences_positions(file_silences)
sil_bef, sil_aft=silences_word_position(word, beg_word, end_word, beg_sil, end_sil, dur_sil, mid_sil)

print(len(sil_bef), len(sil_aft))

7943 7943


##### Speaker change detection

In [6]:
beg_chg_spk, end_chg_spk = read_change_spk(file_spk_change)
print(len(beg_chg_spk), len(end_chg_spk))
n_round, beg_round, end_round, rank_round, text_round = round_extraction(word, beg_word, end_word, beg_chg_spk, end_chg_spk)

print(len(n_round), len(beg_round), len(end_round), len(rank_round), len(text_round), len(beg_chg_spk), len(end_chg_spk))

119 119
7943 7943 7943 7943 7943 119 119


##### ToNy Begin Of Segment

In [7]:

data_prep_tony(name_meeting, word, rank_round, text_round)
tony=convert_tony_results(file_tony_result, word)

print(len(tony))

7943


##### Gold BOS

In [8]:
if file_gold!="":
    gold=convert_gold(file_gold)
    print(len(gold))

7943


##### Ponctuation

In [9]:
prob_nothing, prob_point, prob_comma, prob_nothing_bef, prob_point_bef, prob_comma_bef = convert_punctuation(file_punct, word)
print(len(prob_nothing))


#ponctuation samir

punct_bef, punct=punctuation_samir_extraction(file_punct_samir, word)

exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
7943
7943 7943


##### Audio Features

In [10]:
pitch, pitch_bef, energy, energy_bef = audio_features_extraction(file_pitchenergy, word)

exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
7943 7943 7943 7943


In [11]:
#print(real_rank_round)
#print(rank_round)

In [12]:


vp=0
fn=0
fp=0
vn=0


for i in range(0,len(rank_round)):
    if rank_round[i]==0 and real_rank_round[i]==0:
        vp+=1
    elif rank_round[i]!=0 and real_rank_round[i]==0:
        fn+=1
    elif rank_round[i]==0 and real_rank_round[i]!=0:
        fp+=1
    else:
        vn+=1

print(vp, fn, fp, vn)
p=vp/(vp+fp)
r=vp/(vp+fn)
fscore=2*p*r/(p+r)

print("precision=", p, "rappel=", r, "fscore=", fscore)

42 137 67 7697
precision= 0.3853211009174312 rappel= 0.2346368715083799 fscore= 0.2916666666666667


In [19]:
import pandas as pd
df_dev = pd.DataFrame({'word': word, 'beg_word': beg_word, 'end_word': end_word,'n_round': n_round,
                       'beg_round': beg_round, 'end_round': end_round, 'rank_round' : rank_round,
                       'text_round' : text_round, 'sil_bef' : sil_bef, 'sil_aft' : sil_aft, 'tony' : tony,
                       'gold': gold, 'prob_nothing' : prob_nothing, 'prob_point': prob_point,
                       'prob_comma' : prob_comma, 'prob_nothing_bef' : prob_nothing_bef,
                       'prob_point_bef': prob_point_bef, 'prob_comma_bef' : prob_comma_bef,
                       'real_rank_round': real_rank_round, 'pitch': pitch, 'energy': energy,
                       'pitch_bef': pitch_bef, 'energy_bef': energy_bef})
    

In [24]:
#df_dev.head()
print(prob_point)
for e in prob_point:
    if type(e)==str:
        print(e)

[0, 0, '0.292578', '0.038644', '0.135106', '0.164373', '0.076767', '0.055446', '0.452076', '0.296491', '0.035400', '0.341595', '0.573636', '0.657008', '0.502406', '0.122927', '0.000872', '0.204486', '0.033979', '0.000098', '0.000264', '0.000833', '0.015731', '0.274984', '0.020722', '0.000287', '0.009330', '0.000074', '0.002303', '0.015087', '0.000264', '0.014199', '0.029675', '0.120910', '0.000053', '0.002688', '0.003246', '0.000188', '0.002438', '0.855995', '0.345925', '0.000766', '0.064773', '0.000983', '0.002954', '0.000154', '0.002206', '0.796836', '0.011046', '0.000032', '0.000226', '0.412857', '0.051966', '0.000067', '0.001888', '0.000200', '0.176103', '0.044818', '0.044722', '0.000150', '0.003540', '0.017417', '0.130729', '0.000238', '0.316097', '0.001017', '0.184109', '0.000272', '0.080601', '0.000009', '0.000021', '0.009949', '0.001540', '0.008932', '0.207428', '0.269701', '0.000041', '0.032417', '0.360510', '0.045567', '0.000013', '0.000604', '0.000426', '0.373019', '0.169961

0.003501
0.186529
0.038320
0.072414
0.000144
0.009058
0.001643
0.000163
0.043430
0.033010
0.021785
0.001733
0.000686
0.000223
0.004500
0.033762
0.336779
0.043327
0.003511
0.041725
0.100999
0.000088
0.345596
0.003495
0.000361
0.023808
0.738570
0.093559
0.000025
0.193625
0.000029
0.000105
0.010355
0.005147
0.178077
0.045193
0.273120
0.001290
0.279032
0.000799
0.004967
0.051356
0.049845
0.191194
0.000052
0.127918
0.000158
0.014106
0.261523
0.487155
0.000361
0.062486
0.000022
0.004152
0.007486
0.000767
0.001912
0.000124
0.002080
0.046876
0.233327
0.002007
0.123816
0.008186
0.000150
0.008308
0.000373
0.000040
0.158371
0.000008
0.000163
0.000016
0.000289
0.121681
0.000232
0.137418
0.000014
0.198175
0.000629
0.001707
0.018704
0.043576
0.185828
0.166797
0.010765
0.002235
0.003861
0.054323
0.000237
0.076873
0.000020
0.002568
0.000641
0.000138
0.055419
0.000566
0.000414
0.466875
0.000408
0.000018
0.011418
0.000005
0.000068
0.000233
0.001590
0.000003
0.000320
0.004653
0.000112
0.090653
0.620526
0

0.000032
0.001306
0.002037
0.218017
0.000973
0.000163
0.001327
0.000179
0.016088
0.221102
0.000036
0.015549
0.000088
0.000197
0.027248
0.484803
0.000108
0.003224
0.044283
0.009554
0.001884
0.020032
0.000368
0.000269
0.222256
0.000537
0.000481
0.001638
0.000182
0.001870
0.000111
0.001250
0.000532
0.121492
0.000070
0.165950
0.000099
0.027059
0.001210
0.000776
0.017754
0.005512
0.000236
0.641320
0.000688
0.149609
0.000022
0.000556
0.015465
0.001106
0.281369
0.000021
0.000087
0.002754
0.002485
0.083945
0.004203
0.015812
0.009385
0.000284
0.483223
0.001226
0.000240
0.022544
0.008466
0.213658
0.003474
0.000150
0.000488
0.000150
0.000051
0.002878
0.000106
0.007993
0.000045
0.001515
0.001075
0.185328
0.007857
0.000106
0.000596
0.238572
0.000517
0.041595
0.002313
0.000758
0.003632
0.000774
0.000352
0.404393
0.011119
0.009527
0.000110
0.000238
0.000977
0.004591
0.000390
0.000042
0.106450
0.003472
0.000121
0.000118
0.001814
0.220993
0.001319
0.016471
0.052920
0.173336
0.001881
0.000346
0.042641
0

0.000491
0.005665
0.014368
0.482383
0.000856
0.001070
0.000092
0.049467
0.094515
0.312086
0.000399
0.101242
0.002823
0.000156
0.001163
0.000214
0.348787
0.000056
0.002200
0.000295
0.000115
0.122203
0.174828
0.000106
0.077187
0.028643
0.054999
0.034175
0.276401
0.000017
0.022194
0.137922
0.000013
0.001794
0.000639
0.000460
0.011759
0.000300
0.003416
0.028126
0.000223
0.163773
0.022835
0.102294
0.003311
0.003709
0.014851
0.000038
0.059147
0.004623
0.008020
0.616551
0.001074
0.000241
0.003946
0.334592
0.277473
0.012471
0.479912
0.000196
0.075969
0.000644
0.563314
0.014192
0.000282
0.000113
0.001033
0.002140
0.818244
0.022750
0.066121
0.000094
0.001921
0.013286
0.000033
0.006347
0.000128
0.000314
0.034656
0.004951
0.000613
0.023475
0.350528
0.290200
0.001057
0.316165
0.000031
0.000350
0.000934
0.243153
0.802462
0.168258
0.195267
0.000452
0.003627
0.000010
0.026206
0.206071
0.000678
0.875427
0.001636
0.000018
0.000365
0.071316
0.000586
0.045676
0.000636
0.000388
0.000021
0.000346
0.092064
0

0.002588
0.000226
0.007907
0.648843
0.000082
0.000113
0.001271
0.000135
0.005391
0.000498
0.000218
0.002610
0.000073
0.019234
0.560745
0.001147
0.003203
0.000332
0.000123
0.004045
0.000040
0.004876
0.001468
0.017069
0.004165
0.002753
0.000058
0.086310
0.000261
0.000063
0.000575
0.067023
0.069040
0.000123
0.000536
0.203328
0.002446
0.000240
0.116393
0.000395
0.000007
0.000578
0.041756
0.000565
0.009445
0.001780
0.000258
0.124024
0.000152
0.000042
0.006866
0.000031
0.002963
0.011273
0.000276
0.046034
0.020300
0.000527
0.000057
0.000171
0.096747
0.003978
0.000022
0.000865
0.003151
0.002358
0.039146
0.000081
0.000003
0.000496
0.420038
0.002712
0.000504
0.002996
0.000859
0.699669
0.138557
0.211512
0.116145
0.170376
0.002729
0.000038
0.000055
0.013533
0.013368
0.000051
0.000019
0.000194
0.000919
0.000401
0.007994
0.000149
0.002005
0.000040
0.004561
0.000094
0.301887
0.000161
0.006992
0.000025
0.009186
0.021545
0.000120
0.004199
0.000095
0.017321
0.002019
0.071153
0.000360
0.001251
0.000199
0

0.209578
0.000509
0.002984
0.000130
0.243047
0.000006
0.000064
0.186383
0.000079
0.000981
0.001580
0.000213
0.262565
0.001480
0.000681
0.003282
0.007448
0.000759
0.269566
0.000378
0.001152
0.001243
0.000345
0.470239
0.000003
0.000136
0.019497
0.003293
0.000679
0.000229
0.004838
0.000021
0.001314
0.000267
0.141554
0.000019
0.002839
0.000943
0.000866
0.000098
0.000211
0.003685
0.000605
0.463730
0.000882
0.000020
0.002404
0.000943
0.000866
0.000151
0.000472
0.006124
0.000555
0.019039
0.615133
0.007476
0.040796
0.000012
0.000902
0.226189
0.001500
0.004389
0.001499
0.080657
0.000025
0.002339
0.000328
0.143608
0.000039
0.000766
0.112604
0.000070
0.002638
0.000274
0.093265
0.000341
0.045405
0.000012
0.000485
0.000267
0.026946
0.004420
0.000045
0.000090
0.006004
0.000058
0.003211
0.011767
0.008659
0.000243
0.003680
0.004440
0.000059
0.000708
0.020874
0.000150
0.004645
0.096218
0.398881
0.000002
0.000160
0.000043
0.000769
0.010861
0.703585
0.000002
0.008007
0.000672
0.571611
0.548910
0.001409
0

In [27]:
#df_dev=
file=open("spk_change_comparison_R1.txt", "w")


i=0
liste_texte=[]

for x in df_dev.itertuples():
    mot=x.word
    if x.real_rank_round==0 :
        if float(x.prob_point_bef)<0.5 and i>0 and float(x.prob_comma_bef)<0.5:
            liste_texte.append(".")
        liste_texte.append("\n"+"\n"+"---Real turn---"+"\n"+"\n")
        

    if x.rank_round==0:
        liste_texte.append("|D_SPK|")
    
    #if label[i]==1:
        #liste_texte.append("|D_BOS|")
        
    
    
    if float(x.prob_point_bef)>0.5 or x.real_rank_round==0:
        mot=str(mot).capitalize()
    
    if float(x.prob_point)>0.5:
        mot=mot+"."

    if float(x.prob_comma)>0.5:
        mot=mot+","

    liste_texte.append(mot)
    
    
    

        
    i+=1
    if i==len(df_dev):
        liste_texte.append(".")

file.write(" ".join(liste_texte))
print("bonjou")
file.close()

bonjou


In [15]:
file=open("real_rounds.txt", "w")

for i in range(0,len(real_rank_round)):
    if real_rank_round[i]==0:
        file.write(str(round(real_beg_round[i],3))+" "+str(round(real_end_round[i],3))+" "+real_loc[i]+"\n")

file.close()
        
    

## Création de la dataframe globale avec toutes les réunions

In [2]:
import pandas as pd

liste_dataframes=[]
name_meeting_list =['Linagora_P1', 'Linagora_C1', 'Linagora_R1', 'Linagora_A1', 'Linagora_P6', 'Linagora_C3', 'Linagora_P5', 'Linagora_R4', 'Linagora_R3', 'Linagora_C2', 'Linagora_P4']

for i in range(0,len(name_meeting_list)):
    # fichiers
    print(name_meeting_list[i])
    file_alignments="data/alignments/"+name_meeting_list[i]+"_align.txt"
    file_spk_change="data/speakers/"+name_meeting_list[i]+"_spk.txt"
    file_silences="data/silences/"+name_meeting_list[i]+"_sil.txt"
    file_tony_result="data/tony/"+name_meeting_list[i]+"_tony.txt.split.tok"
    gold_files=['Linagora_P1', 'Linagora_C1', 'Linagora_R1', 'Linagora_A1']
    file_punct="data/punctuation/"+name_meeting_list[i]+"_punct.txt"
    file_pitchenergy="data/audio/"+name_meeting_list[i]+"_pitchenergy.txt"
    file_punct_samir="data/punctuation_samir/"+name_meeting_list[i]+"_samir.txt"
    # calculs
    word,beg_word,end_word = alignments_word_extraction(file_alignments)
    
    real_n_round, real_beg_round, real_end_round, real_rank_round, real_text_round, real_loc=alignments_real_round_extraction(file_alignments)
    
    beg_sil,end_sil,dur_sil,mid_sil=extract_silences_positions(file_silences)
    sil_bef, sil_aft=silences_word_position(word, beg_word, end_word, beg_sil, end_sil, dur_sil, mid_sil)
    beg_chg_spk, end_chg_spk = read_change_spk(file_spk_change)
    n_round, beg_round, end_round, rank_round, text_round = round_extraction(word, beg_word, end_word, beg_chg_spk, end_chg_spk)
    tony=convert_tony_results(file_tony_result, word)
    prob_nothing, prob_point, prob_comma, prob_nothing_bef, prob_point_bef, prob_comma_bef = convert_punctuation(file_punct, word)
    pitch, pitch_bef, energy, energy_bef = audio_features_extraction(file_pitchenergy, word)
    punct, punct_bef= punctuation_samir_extraction(file_punct_samir, word)
    
    if name_meeting_list[i] in gold_files:
        file_gold="data/gold/"+name_meeting_list[i]+"_gold.txt"
        gold=convert_gold(file_gold)
    else:
        gold=[0]*len(word)

    df = pd.DataFrame({'word': word, 'beg_word': beg_word, 'end_word': end_word,'n_round': n_round,
                       'beg_round': beg_round, 'end_round': end_round, 'rank_round' : rank_round,
                       'text_round' : text_round, 'sil_bef' : sil_bef, 'sil_aft' : sil_aft, 'tony' : tony,
                       'gold': gold, 'prob_nothing' : prob_nothing, 'prob_point': prob_point,
                       'prob_comma' : prob_comma, 'prob_nothing_bef' : prob_nothing_bef,
                       'prob_point_bef': prob_point_bef, 'prob_comma_bef' : prob_comma_bef,
                       'real_rank_round': real_rank_round, 'pitch': pitch, 'energy': energy,
                       'pitch_bef': pitch_bef, 'energy_bef': energy_bef, "punct": punct, "punct_bef":punct_bef})
    
    
    df["file"]=name_meeting_list[i]
    liste_dataframes.append(df)
                       
df_all=pd.concat(liste_dataframes)

df_all.head()
df_all.to_csv("df_all_10022021.csv")
     
    

Linagora_P1
7235 7235 7235 7235 7235
exception
exception
exception_a
exception_a
7235 7235 7235 7235
7235 7235
Linagora_C1
1598 1598 1598 1598 1598
1598 1598 1598 1598
1598 1598
Linagora_R1
7943 7943 7943 7943 7943
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
7943 7943 7943 7943
7943 7943
Linagora_A1
1809 1809 1809 1809 1809
1809 1809 1809 1809
ERROR: 1795 à 1804 bon
ERROR2
1809 1809
Linagora_P6
9441 9441 9441 9441 9441
exception
exception
exception
exception
exception
exception
exception_a
exception_a
exception_a
exception_a
exception_a
exception_a
9441 9441 9441 9441
9441 9441
Linagora_C3
1732 1732 1732 1732 1732
1732 1732 1732 1732
1732 1732
Linagora_P5
5720 5720 5720 5720 5720
exception
exception
exception
exception
excepti

In [3]:
df_all.head()
#len(df_all)

,word,beg_word,end_word,n_round,beg_round,end_round,rank_round,text_round,sil_bef,sil_aft,...,prob_point_bef,prob_comma_bef,real_rank_round,pitch,energy,pitch_bef,energy_bef,punct,punct_bef,file
0,ok,2.00,2.37,0,0.0,10.123,0,ok donc juste comme on enregistre euh donc là ...,1.26,0.00,...,0,0,0,0,0,0,0,0,0,Linagora_P1
1,donc,2.73,3.01,0,0.0,10.123,1,ok donc juste comme on enregistre euh donc là ...,0.00,0.24,...,0,0,1,0,0,0,0,0,0,Linagora_P1
2,juste,3.38,3.67,0,0.0,10.123,2,ok donc juste comme on enregistre euh donc là ...,0.24,0.00,...,0,0,2,U,D,0,0,0,0,Linagora_P1
3,comme,3.92,4.22,0,0.0,10.123,3,ok donc juste comme on enregistre euh donc là ...,0.00,0.00,...,0.018153,0.286862,3,D,D,U,D,0,0,Linagora_P1
4,on,4.23,4.67,0,0.0,10.123,4,ok donc juste comme on enregistre euh donc là ...,0.00,0.12,...,0.000417,0.001607,4,D,U,D,D,0,0,Linagora_P1


In [5]:
#df_all.to_csv("df_all.csv")

In [6]:
# création des fichiers pour samir
name_meeting_list =['Linagora_P1', 'Linagora_C1', 'Linagora_R1', 'Linagora_A1', 'Linagora_P6', 'Linagora_C3', 'Linagora_P5', 'Linagora_R4', 'Linagora_R3', 'Linagora_C2', 'Linagora_P4']

for i in range(0,len(liste_dataframes)):
    df=liste_dataframes[i]
    new_file=open("samir/"+name_meeting_list[i]+"_samir.txt", "w")
    liste_mot=[]
    for x in df.itertuples():
        liste_mot.append(x.word)
    
    new_file.write(" ".join(liste_mot))
    new_file.close()
        